In [1]:
%maven org.nd4j:nd4j-cuda-9.0:1.0.0-beta
%maven org.deeplearning4j:deeplearning4j-cuda-9.0:1.0.0-beta
%maven org.datavec:datavec-api:1.0.0-beta

%maven org.datavec:datavec-spark_2.11:1.0.0-beta_spark_1
%maven org.deeplearning4j:dl4j-spark_2.11:1.0.0-beta_spark_1
%maven org.knowm.xchart:xchart:3.5.2
    
%maven org.scala-lang:scala-library:2.13.0-M5-6e0cba7

In [2]:
import org.deeplearning4j.nn.graph.ComputationGraph;
import org.deeplearning4j.nn.transferlearning.TransferLearning;
import org.deeplearning4j.nn.api.OptimizationAlgorithm;
import org.deeplearning4j.nn.weights.WeightInit;
import org.deeplearning4j.nn.conf.*;
import org.deeplearning4j.nn.conf.layers.*;
import org.deeplearning4j.nn.conf.graph.rnn.*;
import org.deeplearning4j.nn.conf.inputs.InputType;
import org.deeplearning4j.nn.conf.WorkspaceMode;
import org.deeplearning4j.optimize.listeners.ScoreIterationListener;
import org.deeplearning4j.datasets.iterator.MultipleEpochsIterator;
import org.deeplearning4j.datasets.datavec.RecordReaderMultiDataSetIterator;
import org.deeplearning4j.util.ModelSerializer;
;
import org.datavec.api.transform.*;
import org.datavec.api.transform.transform.time.StringToTimeTransform;
import org.datavec.api.transform.sequence.comparator.NumericalColumnComparator;
import org.datavec.api.transform.transform.string.ConcatenateStringColumns;
import org.datavec.api.transform.transform.doubletransform.MinMaxNormalizer;
import org.datavec.api.transform.schema.Schema;
import org.datavec.api.transform.metadata.StringMetaData;
import org.datavec.api.records.reader.impl.csv.CSVRecordReader;
import org.datavec.api.split.FileSplit;
import org.datavec.spark.storage.SparkStorageUtils;
import org.datavec.spark.transform.misc.StringToWritablesFunction;
import org.datavec.spark.transform.SparkTransformExecutor;
import org.datavec.api.transform.condition.*;
import org.datavec.api.transform.condition.column.*;
import org.datavec.api.transform.sequence.window.ReduceSequenceByWindowTransform;
import org.datavec.api.transform.reduce.Reducer;
import org.datavec.api.transform.reduce.AggregableColumnReduction;
import org.datavec.api.transform.sequence.window.TimeWindowFunction;
import org.datavec.api.transform.ops.IAggregableReduceOp;
import org.datavec.api.transform.metadata.ColumnMetaData;
import org.datavec.api.writable.*;
import org.datavec.hadoop.records.reader.mapfile.MapFileSequenceRecordReader;
//import org.datavec.api.util.ArchiveUtils;


import org.nd4j.util.ArchiveUtils 


;
import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.dataset.api.MultiDataSetPreProcessor;
import org.nd4j.linalg.dataset.api.MultiDataSet;
import org.nd4j.linalg.lossfunctions.LossFunctions;
import org.nd4j.linalg.activations.Activation;
import org.nd4j.linalg.learning.config.*;
import org.nd4j.linalg.factory.Nd4j;
import org.nd4j.linalg.indexing.BooleanIndexing;
import org.nd4j.linalg.indexing.INDArrayIndex;
import org.nd4j.linalg.indexing.NDArrayIndex.*;
import org.nd4j.linalg.indexing.conditions.Conditions;
;
import org.apache.spark.api.java.function.Function;
import org.apache.commons.io.FileUtils;
import org.joda.time.DateTimeZone;
import org.joda.time.format.DateTimeFormat;
;
import scala.collection.JavaConversions.*;
import scala.collection.JavaConverters.*;
import scala.io.Source;
import java.util.Random;
import java.util.concurrent.TimeUnit;
import java.io.*;
import java.net.URL;

File cache = new File(System.getProperty("user.home"), "/.deeplearning4j");


In [3]:
public File downloadFile(File cache, String url) throws Exception {
    String splitPath[] = url.split("/");
    String fileName = splitPath[splitPath.length-1];
    System.out.println(fileName);
    File tmpZip = new File(cache, fileName);
    tmpZip.delete();
    System.out.println("Downloading file...");
    FileUtils.copyURLToFile(new URL(url), tmpZip);
    System.out.println("Finished Downloading.");
    return tmpZip;
}

In [4]:
File dataFile = new File(cache, "/aisdk_20171001.csv");
if (!dataFile.exists()) {
    String remote = "http://blob.deeplearning4j.org/datasets/aisdk_20171001.csv.zip";
    File tmpZip = downloadFile(cache, remote);
    System.out.println("Decompressing file...");
    ArchiveUtils.unzipFileTo(tmpZip.getAbsolutePath(), cache.getAbsolutePath());
    tmpZip.delete();
    System.out.println("Done.");
} else {
    System.out.println("File already Exists.");
}

File already Exists.


In [5]:
/*Don't Run This part twice*/
import org.apache.spark.SparkContext ;
import org.apache.spark.sql.SQLContext ;
import org.apache.spark.SparkConf ;

SparkConf conf = new SparkConf().setAppName("T2P App").setMaster("local[2]");
SQLContext sqlContext = new SQLContext(new SparkContext(conf))


[IJava-executor-0] INFO org.apache.spark.SparkContext - Running Spark version 1.6.3
[IJava-executor-0] WARN org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[IJava-executor-0] WARN org.apache.spark.util.Utils - Your hostname, phoenix resolves to a loopback address: 127.0.1.1; using 10.33.9.66 instead (on interface eno1)
[IJava-executor-0] WARN org.apache.spark.util.Utils - Set SPARK_LOCAL_IP if you need to bind to another address
[IJava-executor-0] INFO org.apache.spark.SecurityManager - Changing view acls to: bzr0014
[IJava-executor-0] INFO org.apache.spark.SecurityManager - Changing modify acls to: bzr0014
[IJava-executor-0] INFO org.apache.spark.SecurityManager - SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(bzr0014); users with modify permissions: Set(bzr0014)
[IJava-executor-0] INFO io.netty.util.internal.PlatformDependent - Your platform do

In [7]:
sqlContext.read()
    .format("com.databricks.spark.csv")
    .option("header", "true") // Use first line of all files as header
    .option("inferSchema", "true") // Automatically infer data types
    .load(dataFile.getAbsolutePath())

EvalException: Failed to find data source: com.databricks.spark.csv. Please find packages at http://spark-packages.org